# 编写第一个应用polls

在这个投票应用中，将有以下四个视图：  
* Question首页 —— 显示最新发布的几个Question。
* Question“详细”页面 —— 显示单个Question的具体内容，不显示该议题的当前投票结果，而是提供一个投票的表单。
* Question“结果”页面 —— 显示特定的Question的投票结果。
* 投票功能 —— 处理对Question中Choice的投票。

## 编辑流程（以第一个视图为例）

视图在polls/views.py中创建

In [ ]:
from django.http import HttpResponse
def index(request):
    return HttpResponse("Hello, world. 这里是投票界面首页。")

把投票应用的视图映射都URL上  
这需要在投票应用的目录内部创建URLconf,即创建一个urls.py文件。  
urls.py用于写入不同页面所对应的功能，名称

In [ ]:
polls/
    __init__.py
    admin.py
    models.py
    tests.py
    urls.py
    views.py

在polls/urls.py中写入以下代码：

In [ ]:
from django.conf.urls import url

from . import views  #导入视图

urlpatterns = [
    url(r'^$', views.index, name='index'),  #正则表达式，视图参数，命名
]

让主URLconf链接到polls.urls模块中。  
这需要在project01/urls.py中插入一个include():

In [ ]:
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^polls/', include('polls.urls')),  #插入的内容，第一个参数是正则表达式，第二个是视图参数
    url(r'^admin/', include(admin.site.urls)),
]

## 编辑其它视图

在polls/views.py添加一些更多的视图

In [ ]:
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)

将这些新的视图和polls.urls模块关联起来  
编辑polls/urls.py

In [ ]:
from django.conf.urls import url

from . import views

urlpatterns = [
    # ex: /polls/
    url(r'^$', views.index, name='index'),
    # ex: /polls/5/
    url(r'^(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
    # ex: /polls/5/results/
    url(r'^(?P<question_id>[0-9]+)/results/$', views.results, name='results'),
    # ex: /polls/5/vote/
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]

|地址|指向|
|----|----|
|/polls/|index页面|
|/polls/123/|detail页面|
|/polls/123/results/|results页面|
|/polls/123/vote/|vote页面|

## 编辑页面的功能

### 使index页码显示最新的5条Question

修改polls/views.py的index部分

In [ ]:
from django.http import HttpResponse

from .models import Question  #需要导入模型中的Question


def index(request):
    latest_question_list = Question.objects.order_by('-publish_date')[:5]  #显示最新的五条问题
    output = ', '.join([p.question_text for p in latest_question_list])
    return HttpResponse(output)

## 使用模板

在polls/下创建templates文件夹  
在polls/templates/下建立polls文件夹  
在创建一个index.html文件,写入

In [ ]:
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}

更新polls/views.py下的index视图

In [ ]:
from django.http import HttpResponse
from django.template import RequestContext, loader

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')  #加载模板
    context = RequestContext(request, {
        'latest_question_list': latest_question_list,
    })
    return HttpResponse(template.render(context))

### 使用模板的快捷方式

把polls/views.py下的index修改为

In [ ]:
from django.shortcuts import render

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)

## 引发404错误

如果没有找到所请求ID的Question（即detail页面），这个视图引发一个Http404异常  
在polls/views.py修改detail

In [ ]:
from django.http import Http404
from django.shortcuts import render

from .models import Question
# ...
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, 'polls/detail.html', {'question': question})

### 引发404错误的快捷方式

在polls/views.py修改detail

In [ ]:
from django.shortcuts import get_object_or_404, render

from .models import Question
# ...
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/detail.html', {'question': question})

## 移除模板中硬编码的URLs

修改polls/index.html  
原来是：

In [ ]:
<li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>

修改后：

In [ ]:
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

如果你想把polls应用中detail视图的URL改成其它样子比如 polls/specifics/12/，就可以不必在该模板（或者多个模板）中修改它，只需要修改 polls/urls.py

In [ ]:
...
# added the word 'specifics'
url(r'^specifics/(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
...

## 带命名空间的URL名字

在你的主URLconf下添加命名空间。 在project01/urls.py文件中，添加命名空间将它修改成：

In [ ]:
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^polls/', include('polls.urls', namespace="polls")),
    url(r'^admin/', include(admin.site.urls)),
]

现在将你的模板polls/index.html由：

In [ ]:
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

修改为指向具有命名空间的详细视图：

In [ ]:
<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>